In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,058 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,776 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrame Basics").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/Ekenc/Project4/main/Data/Merged_Target_Glaciers.csv"
spark.sparkContext.addFile(url)
df = pd.read_csv(url)

# Show DataFrame
df.head()


,Unnamed: 0,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Mean cumulative mass balance of glaciers,Precipitation Anomaly,Mean Adjusted Sea Level (inches),Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean)
0,0,1964,291.2,319.620000,1260.30,-2.545,-0.041776,4.169291,-0.546,-0.252
1,1,1967,291.5,322.180000,1284.03,-2.662,-0.096894,4.452756,-0.420,0.000
2,2,1970,293.8,325.620315,1351.70,-3.519,-0.070516,4.677165,-0.294,0.108
3,3,1971,294.0,326.320000,1357.20,-3.758,0.032240,4.881890,-0.510,-0.126
4,4,1972,295.6,328.742110,1380.10,-4.016,-0.772485,5.240157,-0.186,0.072


In [5]:
# Define features set
X = df.drop("Mean cumulative mass balance of glaciers", axis = 1)
X = X.drop("Unnamed: 0", axis = 1)
X= X.drop("Precipitation Anomaly", axis = 1)
X.head()

,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Mean Adjusted Sea Level (inches),Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean)
0,1964,291.2,319.620000,1260.30,4.169291,-0.546,-0.252
1,1967,291.5,322.180000,1284.03,4.452756,-0.420,0.000
2,1970,293.8,325.620315,1351.70,4.677165,-0.294,0.108
3,1971,294.0,326.320000,1357.20,4.881890,-0.510,-0.126
4,1972,295.6,328.742110,1380.10,5.240157,-0.186,0.072


In [6]:
url1 = "https://raw.githubusercontent.com/Ekenc/Project4/main/Data/Linear_Projected_SeaLevel_Dataframe.csv"
df1 = pd.read_csv(url1)

# Show DataFrame
df1.head()

,Unnamed: 0,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Mean Adjusted Sea Level (inches),Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean)
0,0,2020,330.714233,405.576768,1906.690278,9.302441,0.770041,1.612599
1,1,2021,331.466061,407.257998,1916.355223,9.396314,0.792619,1.645512
2,2,2022,332.217889,408.939228,1926.020169,9.490188,0.815197,1.678425
3,3,2023,332.969718,410.620458,1935.685114,9.584061,0.837775,1.711339
4,4,2024,333.721546,412.301688,1945.350059,9.677934,0.860353,1.744252


In [7]:
df1=df1.drop("Unnamed: 0", axis = 1)
df1.head()

,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Mean Adjusted Sea Level (inches),Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean)
0,2020,330.714233,405.576768,1906.690278,9.302441,0.770041,1.612599
1,2021,331.466061,407.257998,1916.355223,9.396314,0.792619,1.645512
2,2022,332.217889,408.939228,1926.020169,9.490188,0.815197,1.678425
3,2023,332.969718,410.620458,1935.685114,9.584061,0.837775,1.711339
4,2024,333.721546,412.301688,1945.350059,9.677934,0.860353,1.744252


In [8]:
# Define target vector
y = df["Mean cumulative mass balance of glaciers"]
y[:48]

0     -2.545
1     -2.662
2     -3.519
3     -3.758
4     -4.016
5     -4.147
6     -4.339
7     -4.534
8     -5.140
9     -5.919
10    -6.726
11    -7.009
12    -7.586
13    -7.475
14    -7.559
15    -7.892
16    -8.399
17    -8.832
18    -8.935
19    -9.242
20    -9.917
21   -10.384
22   -10.819
23   -11.611
24   -12.134
25   -12.861
26   -13.088
27   -13.317
28   -13.755
29   -14.326
30   -15.018
31   -15.758
32   -16.591
33   -17.202
34   -17.619
35   -18.169
36   -19.011
37   -19.920
38   -20.657
39   -21.502
40   -22.285
41   -23.402
42   -24.383
43   -25.152
44   -26.043
45   -27.174
Name: Mean cumulative mass balance of glaciers, dtype: float64

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
                                                                                                

In [11]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Create a linear regression model
model = LinearRegression()

In [14]:
# Train the model
model.fit(X_train, y_train)

LinearRegression()

In [15]:
# Make predictions
predictions = model.predict(X_test)

In [16]:
# Calculate the mean squared error, model evaluation
print(
  'mean_squared_error : ', mean_squared_error(y_test, predictions))
print(
  'mean_absolute_error : ', mean_absolute_error(y_test, predictions))


# Compute the linear regression model score
model.score(X_test, y_test)


mean_squared_error :  0.12576685019343228
mean_absolute_error :  0.304382725895586


0.9964936797077786

In [17]:
# Display features
model.feature_names_in_

array(['Year', 'Mean Nitrous Oxide Concentration',
       'Mean Carbon Dioxide Concentration', 'Mean Methane Concentration',
       'Mean Adjusted Sea Level (inches)', 'Sea Temperature Anomaly',
       'Earth Surface Temperature Anomaly (land and ocean)'], dtype=object)

In [18]:
# Display the slope
print(f"Model's slope: {model.coef_}")

Model's slope: [ 0.4646149  -0.37305732 -0.41515923  0.00394074  0.18784548 -1.52530308
  0.97114348]


In [19]:
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

Model's y-intercept: -680.4605830963199


In [20]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's formula: y = -680.4605830963199 + 0.464614902102358X


In [21]:
# Display the formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]} * 2030")

# Predict the 
y_0 = model.intercept_ + model.coef_[0] *2030

# Display the prediction
print(f": {y_0:.3f}")

Model's formula: y = -680.4605830963199 + 0.464614902102358 * 2030
: 262.708


In [23]:
#Check Model with Predictions data
Glaciers_Predictions = model.predict(df1)
Glaciers_Predictions

array([-24.04006506, -24.50065751, -24.96124997, -25.42184242,
       -25.88243488, -26.34302733, -26.80361979, -27.26421224,
       -27.7248047 , -28.18539716, -28.64598961, -29.10658207,
       -29.56717452, -30.02776698, -30.48835943, -30.94895189,
       -31.40954434, -31.8701368 , -32.33072925, -32.79132171,
       -33.25191417, -33.71250662, -34.17309908, -34.63369153,
       -35.09428399, -35.55487644, -36.0154689 , -36.47606135,
       -36.93665381, -37.39724627, -37.85783872])

In [24]:
Glaciers_Predictions_df=df1.copy()
Glaciers_Predictions_df["Mean cumulative mass balance of glaciers"] = Glaciers_Predictions
Glaciers_Predictions_df = Glaciers_Predictions_df[['Year','Mean Nitrous Oxide Concentration', 'Mean Carbon Dioxide Concentration', 'Mean Methane Concentration', 'Mean Adjusted Sea Level (inches)', 'Sea Temperature Anomaly', 'Earth Surface Temperature Anomaly (land and ocean)','Mean cumulative mass balance of glaciers']]
Glaciers_Predictions_df

,Year,Mean Nitrous Oxide Concentration,Mean Carbon Dioxide Concentration,Mean Methane Concentration,Mean Adjusted Sea Level (inches),Sea Temperature Anomaly,Earth Surface Temperature Anomaly (land and ocean),Mean cumulative mass balance of glaciers
0,2020,330.714233,405.576768,1906.690278,9.302441,0.770041,1.612599,-24.040065
1,2021,331.466061,407.257998,1916.355223,9.396314,0.792619,1.645512,-24.500658
2,2022,332.217889,408.939228,1926.020169,9.490188,0.815197,1.678425,-24.961250
3,2023,332.969718,410.620458,1935.685114,9.584061,0.837775,1.711339,-25.421842
4,2024,333.721546,412.301688,1945.350059,9.677934,0.860353,1.744252,-25.882435
5,2025,334.473374,413.982918,1955.015005,9.771808,0.882931,1.777165,-26.343027
6,2026,335.225202,415.664148,1964.679950,9.865681,0.905509,1.810079,-26.803620
7,2027,335.977030,417.345378,1974.344895,9.959554,0.928088,1.842992,-27.264212
8,2028,336.728858,419.026608,1984.009840,10.053428,0.950666,1.875905,-27.724805
9,2029,337.480687,420.707838,1993.674786,10.147301,0.973244,1.908819,-28.185397


In [26]:
from google.colab import files
Glaciers_Predictions_df.to_csv("Linear_Glacier_Dataframe.csv",encoding = 'utf-8-sig')
files.download('Linear_Glacier_Dataframe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>